# Feature Extraction

### TODO

- Create the Tables for Authors, Venues
- Perform Feature Extractions
- Create the final table with the features

#### Adding the libraries

In [1]:
import pandas as pd
import matplotlib as plt
import ast
import numpy as np
import seaborn as sns
from statistics import mean

#### Adding the database

In [2]:
data_papers = pd.read_pickle('../Data/papers_clean.pkl')
data_papers.head()

,id,authors,title,year,n_citation,page_start,page_end,doc_type,publisher,volume,issue,doi,references,fos,venue,indexed_abstract,alias_ids
0,75067,"[{'name': 'Narciso Cerpa', 'id': 2571766951}, ...",Comparing Estimated with Actual Efforts for a ...,2001,0,348,353,Conference,IIIS,,,,"[1989212489, 2007884971, 2014210483, 202969654...","[{'name': 'Software engineering', 'w': 0.46281...","{'raw': 'World Multiconference on Systemics, C...",NaN,NaN
1,40449,"[{'name': 'José Luis Martín', 'id': 2322634291...",Software Workbench for Movement Estimation bas...,2001,1,517,522,,,,,,NaN,"[{'name': 'Workbench', 'w': 0.59515}, {'name':...",{'raw': 'VIIP'},NaN,NaN
2,73220,"[{'name': 'John F. Duff', 'id': 2670839461}, {...",Parallel Molecular Dynamics on Scyld Clusters.,2001,0,,,,,,,,NaN,"[{'name': 'Chemical physics', 'w': 0.41215}, {...",{'raw': 'PPSC'},NaN,NaN
3,136660,"[{'name': 'Junji Tomita', 'id': 2097955466}, {...",Interactive Web Search by Graphical Query Refi...,2001,17,,,,,,,,"[1982451429, 1997841190, 2165612380]","[{'name': 'Query optimization', 'w': 0.70923},...",{'raw': 'WWW Posters'},"{'IndexLength': 78, 'InvertedIndex': {'through...",NaN
4,155019,"[{'name': 'Giuseppe Attardi', 'id': 1557751177...",PIQASso: PIsa Question Answering System,2001,46,566,607,Conference,NIST,,,,"[26591655, 1511749302, 1565963236, 2094276686,...","[{'name': 'Logical relations', 'w': 0.0}, {'na...","{'raw': 'Text REtrieval Conference', 'id': 253...","{'IndexLength': 140, 'InvertedIndex': {'tagger...",NaN


#### The list of features:

* Topic Rank
* Diversity
* Productivity
* H-index
* Author Rank
* Venue Rank
* Maximum Past Influence of Authors (Past Influence of Authors)
* Total Past Influence of Authors (Past Influence of Authors)
* Maximum Past Influence of Venues (Past Influence of Venues)
* Total Past Influence of Venues (Past Influence of Venues)
* Versatility
* Novelty
* Sociality
* Authority
* Venue Centrality
* First two years performance 
* Yearly citations 

#### The features that need merge (like multi authors) will be calculated during feature file creation 

#### Topic Rank

steps:

- take the fos of all rows
- take the unique values
- create a --- matrix where the columns are the topics and each row is a paper, if topic exist => 1, otherwise 0
- have a table with the topic as ID
- calculate the score for each ID ==> 
- third column should be the rank

INF(topic/d)= P(topic/d) * INF(paper)
score = sum(Weight of fos in Document * n_citation of that document) in all papers
rank = sort(score,ascending)

Table: topics (topic, score, rank)

In [6]:
topics = data_papers['fos']

#### Diversity

steps:

- loop the FOS sum the Shannon index
- Shannon index = -w Ln (w)

The higher the value the more diversity; The lower the value the more focused the paper is

https://en.wikipedia.org/wiki/Diversity_index

https://en.wikipedia.org/wiki/Entropy_(information_theory)

Table: paper

#### Productivity 

steps (table):

- option 1:
    - each row: [{'name':"name1",'id':"id1", 'org':"org1"},{'name':"name2",'id':, 'org':""},...]
    - a = [['name1','id1','org1'],...]
    - a = list(set(a))
    - a = [['name1','id1','org1', [paper_id1,paper_id2,..]],...]

- option 2:
    - [ [['name1','id1','org1'],[paper_id1]], ...]

- option 3*:
    - loop the papers and make this:
    - {'id1':['name1','org1',[paper_id1,paper_id2,..]],..}

steps (feature):
- len(papers)
- calculate the final value during the feature file creation

Table: Author (id, name, org, papers, productivity)

#### H-index 

steps:
- use the code from before
- calculate the final value during the feature file creation

Table: Author (id, name, org, papers, productivity, h_index)

#### Author Rank

steps:
- get citation of the papers the author has
- take average
- give rank 

Table: Author (id, name, org, papers, productivity, h_index, citations, ave_cite, author_rank)

#### Venue Rank

steps (table):
- same way as author
- loop the papers and make this:
- {'id1':['raw1',[paper_id1,paper_id2,..]],..}


steps (feature):
- get citation of the papers the venue has
- take average
- give rank


table: Venue (id, raw (or name), papers(list of ids), citations, ave_citation, rank)

#### Maximum Past Influence of Authors (Past Influence of Authors)

steps:
- max of citation

Table: Author (id, name, org, papers, productivity, h_index, citations, ave_cite, author_rank, author_MPI)

#### Total Past Influence of Authors (Past Influence of Authors)

steps:
- total citation

Table: Author (id, name, org, papers, productivity, h_index, citations, ave_cite, author_rank, author_MPI, author_TPI)

#### Maximum Past Influence of Venue (Past Influence of Venue)

steps:
- max of citation

table: Venue (id, raw (or name), papers(list of ids), citations, ave_citation, rank, venur_MPI)

#### Total Past Influence of Venue (Past Influence of Venue)

steps:
- total citation

table: Venue (id, raw (or name), papers(list of ids), citations, ave_citation, rank, venur_MPI, venur_TPI)

#### Versatility 

steps:
- add column topics to table author (format: {'FOS1':average w, 'FOS2':average w, ...})
- go over the FOS of papers of author
    - if FOS in dic, value = value + w/productivity
    - if not, create new key with FOS name and value FOS w/productivity
- what we get per row {'FOS1':average w, 'FOS2':average w, ...}
- loop the FOS sum the Shannon index
- Shannon index = -w Ln (w)


Table: Author (id, name, org, papers, productivity, h_index, citations, ave_cite, author_rank, author_MPI, author_TPI, versatility)